**Pytorch를 이용한 MNIST 학습** 

**Training Data 로드**

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils as utils

from torchvision import datasets    # 데이터
from torchvision import transforms  # 네트워크 모델 전처리

transform = transforms.Compose([
    transforms.ToTensor(), 
    transforms.Normalize((0.5,), (0.5,))])

trainset    = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
trainloader = utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

testset     = datasets.MNIST(root='./data', train=False, download=True, transform=transform)
testloader  = utils.data.DataLoader(testset, batch_size=64, shuffle=False)

**DNN 모델 구성 (Class)**

In [6]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 64 * 7 * 7)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

net = Net()

![Alt text](image-1.png)

**Training** 

In [7]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

for epoch in range(5): 
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data

        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    print(f'Epoch {epoch+1}, Loss: {running_loss/len(trainloader)}')

print('Finished Training')

Epoch 1, Loss: 0.8728671381786179
Epoch 2, Loss: 0.19267425777068906
Epoch 3, Loss: 0.12441571259191994
Epoch 4, Loss: 0.09640894476562611
Epoch 5, Loss: 0.08019169943599797
Finished Training


**Evaluation**

In [8]:
correct = 0
total = 0

with torch.no_grad():
    for data in testloader:
        inputs, labels = data
        outputs = net(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy on the test images: {100 * correct / total}%')

Accuracy on the test images: 97.91%
